In [20]:
from microphone import record_audio
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import librosa
import matplotlib.mlab as mlab
from scipy.ndimage.morphology import generate_binary_structure
from scipy.ndimage.morphology import iterate_structure
from typing import Tuple, List
from numba import njit

In [21]:
def micRecord(time=10):
    frames, rate = record_audio(time)
    return np.hstack([np.frombuffer(i, np.int16) for i in frames]), rate

In [22]:
def getFile(path):
    recorded_audio, sampling_rate = librosa.load(path, 
                                                 sr=44100, 
                                                 mono=True, 
                                                 offset=1.5, 
                                                 duration=20)
    return recorded_audio, sampling_rate

In [23]:
def userinput():
    while True:
        audioType = input("u for Upload, r for Record: ")
        if audioType == 'u':
            path = input("Enter path to file: ")
            samples, rate = getFile(path)
            break
        elif audioType == 'r':
            samples, rate = micRecord()
            break
        print("Invalid input. Try again.")
    # print(rate)
    
    return samples, rate

In [29]:
def ecdf(data):
    """Returns (x) the sorted data and (y) the empirical cumulative-proportion
    of each datum.
    
    Parameters
    ----------
    data : numpy.ndarray, size-N
    
    Returns
    -------
    Tuple[numpy.ndarray shape-(N,), numpy.ndarray shape-(N,)]
        Sorted data, empirical CDF values"""
    data = np.asarray(data).ravel()  # flattens the data
    y = np.linspace(1 / len(data), 1, len(data))  # stores the cumulative proportion associated with each sorted datum
    x = np.sort(data)
    return x, y

In [30]:
@njit
def _peaks(
    data_2d: np.ndarray, nbrhd_row_offsets: np.ndarray, nbrhd_col_offsets: np.ndarray, amp_min: float
) -> List[Tuple[int, int]]:
    """
    A Numba-optimized 2-D peak-finding algorithm.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected.
    nbrhd_row_offsets : numpy.ndarray, shape-(N,)
        The row-index offsets used to traverse the local neighborhood.
        
        E.g., given the row/col-offsets (dr, dc), the element at 
        index (r+dr, c+dc) will reside in the neighborhood centered at (r, c).
    
    nbrhd_col_offsets : numpy.ndarray, shape-(N,)
        The col-index offsets used to traverse the local neighborhood. See
        `nbrhd_row_offsets` for more details.
        
    amp_min : float
        All amplitudes equal to or below this value are excluded from being
        local peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned in 
        column-major order
    """
    peaks = []  # stores the (row, col) locations of all the local peaks

    # Iterating over each element in the the 2-D data 
    # in column-major ordering
    #
    # We want to see if there is a local peak located at
    # row=r, col=c
    for c, r in np.ndindex(*data_2d.shape[::-1]):
        if data_2d[r, c] <= amp_min:
            # The amplitude falls beneath the minimum threshold
            # thus this can't be a peak.
            continue
        
        # Iterating over the neighborhood centered on (r, c) to see
        # if (r, c) is associated with the largest value in that
        # neighborhood.
        #
        # dr: offset from r to visit neighbor
        # dc: offset from c to visit neighbor
        for dr, dc in zip(nbrhd_row_offsets, nbrhd_col_offsets):
            if dr == 0 and dc == 0:
                # This would compare (r, c) with itself.. skip!
                continue

            if not (0 <= r + dr < data_2d.shape[0]):
                # neighbor falls outside of boundary.. skip!
                continue

            if not (0 <= c + dc < data_2d.shape[1]):
                # neighbor falls outside of boundary.. skip!
                continue

            if data_2d[r, c] < data_2d[r + dr, c + dc]:
                # One of the amplitudes within the neighborhood
                # is larger, thus data_2d[r, c] cannot be a peak
                break
        else:
            # if we did not break from the for-loop then (r, c) is a local peak
            peaks.append((r, c))
    return peaks

In [31]:
def local_peak_locations(data_2d: np.ndarray, neighborhood: np.ndarray, amp_min: float):
    """
    Defines a local neighborhood and finds the local peaks
    in the spectrogram, which must be larger than the specified `amp_min`.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected
    
    neighborhood : numpy.ndarray, shape-(h, w)
        A boolean mask indicating the "neighborhood" in which each
        datum will be assessed to determine whether or not it is
        a local peak. h and w must be odd-valued numbers
        
    amp_min : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned
        in column-major ordering.
    
    Notes
    -----
    The local peaks are returned in column-major order, meaning that we 
    iterate over all nbrhd_row_offsets in a given column of `data_2d` in search for
    local peaks, and then move to the next column.
    """

    # We always want our neighborhood to have an odd number
    # of nbrhd_row_offsets and nbrhd_col_offsets so that it has a distinct center element
    assert neighborhood.shape[0] % 2 == 1
    assert neighborhood.shape[1] % 2 == 1
    
    # Find the indices of the 2D neighborhood where the 
    # values were `True`
    #
    # E.g. (row[i], col[i]) stores the row-col index for
    # the ith True value in the neighborhood (going in row-major order)
    nbrhd_row_indices, nbrhd_col_indices = np.where(neighborhood)
    

    # Shift the neighbor indices so that the center element resides 
    # at coordinate (0, 0) and that the center's neighbors are represented
    # by "offsets" from this center element.
    #
    # E.g., the neighbor above the center will has the offset (-1, 0), and 
    # the neighbor to the right of the center will have the offset (0, 1).
    nbrhd_row_offsets = nbrhd_row_indices - neighborhood.shape[0] // 2
    nbrhd_col_offsets = nbrhd_col_indices - neighborhood.shape[1] // 2

    return _peaks(data_2d, nbrhd_row_offsets, nbrhd_col_offsets, amp_min=amp_min)

In [32]:
def find_min_amp(spectrogram, amp_threshold):
    log_S = np.log(spectrogram).ravel()  # flattened array
    ind = round(len(log_S) * amp_threshold)
    cutoff_log_amplitude = np.partition(log_S, ind)[ind]
    return cutoff_log_amplitude

In [33]:
def peak_extract(samples, sampling_rate, *, amp_threshold=0.75, neighborhood_rank=2, neighborhood_connectivity=1, neighborhood_iterations=20):
	
    """
    Extracts peaks from a spectrogram created from the sample data.
    
    Parameters
    ----------
    samples : numpy.ndarray
        Array of audio samples
	
	sampling_rate : int
		The sampling rate of the audio samples
        
    amp_threshold : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
	neighborhood_rank : int
	neighborhood_connectivity : int
	neighborhood_iterations : int
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned
        in column-major ordering.
    """
    
    time = np.arange(len(samples)) / sampling_rate

    base_structure = generate_binary_structure(neighborhood_rank,neighborhood_connectivity)
    neighborhood = iterate_structure(base_structure, neighborhood_iterations)

    spectrogram, freqs, times = mlab.specgram(
		samples,
		NFFT=4096,
		Fs=sampling_rate,
		window=mlab.window_hanning,
		noverlap=int(4096 / 2)
	)

    spectrogram = np.clip(spectrogram, 10**-20, None)
    
    amp_min = find_min_amp(spectrogram, amp_threshold)

    return local_peak_locations(spectrogram, neighborhood, amp_min)

In [34]:
samples, sample_rate = userinput()
peaks = peak_extract(samples, sample_rate)

u for Upload, r for Record: u
Enter path to file: C:\Users\nketi\Downloads\Flashing_Lights_(getmp3.pro).mp3


In [35]:
peaks

[(1280, 0),
 (1689, 0),
 (1773, 0),
 (1976, 0),
 (1001, 1),
 (1164, 1),
 (1318, 1),
 (1396, 1),
 (1727, 1),
 (1805, 1),
 (2017, 1),
 (2044, 1),
 (766, 2),
 (855, 2),
 (955, 2),
 (1093, 2),
 (1249, 2),
 (1574, 2),
 (155, 3),
 (260, 3),
 (346, 3),
 (434, 3),
 (525, 3),
 (1123, 3),
 (1531, 3),
 (232, 4),
 (465, 4),
 (622, 4),
 (697, 4),
 (929, 4),
 (192, 5),
 (310, 5),
 (544, 5),
 (1497, 5),
 (1654, 5),
 (1048, 6),
 (1182, 6),
 (1457, 6),
 (389, 7),
 (1223, 7),
 (1632, 7),
 (77, 8),
 (1424, 13),
 (172, 14),
 (747, 15),
 (2010, 16),
 (649, 18),
 (1354, 18),
 (138, 19),
 (240, 19),
 (275, 19),
 (514, 19),
 (1066, 19),
 (1239, 19),
 (206, 20),
 (413, 20),
 (583, 20),
 (723, 20),
 (984, 20),
 (102, 21),
 (689, 21),
 (884, 21),
 (445, 22),
 (826, 22),
 (1170, 22),
 (1579, 22),
 (7, 25),
 (290, 36),
 (876, 37),
 (122, 39),
 (817, 41),
 (390, 42),
 (349, 44),
 (843, 44),
 (1202, 45),
 (614, 46),
 (1409, 46),
 (82, 47),
 (1064, 47),
 (687, 48),
 (715, 48),
 (1436, 48),
 (310, 49),
 (519, 49),
 (1

In [42]:
peak_dict = {}
for i,v in enumerate(peaks):
    peak_dict.setdefault(peaks[i][1], []).append(peaks[i][0])
peak_dict

{0: [1280, 1689, 1773, 1976],
 1: [1001, 1164, 1318, 1396, 1727, 1805, 2017, 2044],
 2: [766, 855, 955, 1093, 1249, 1574],
 3: [155, 260, 346, 434, 525, 1123, 1531],
 4: [232, 465, 622, 697, 929],
 5: [192, 310, 544, 1497, 1654],
 6: [1048, 1182, 1457],
 7: [389, 1223, 1632],
 8: [77],
 13: [1424],
 14: [172],
 15: [747],
 16: [2010],
 18: [649, 1354],
 19: [138, 240, 275, 514, 1066, 1239],
 20: [206, 413, 583, 723, 984],
 21: [102, 689, 884],
 22: [445, 826, 1170, 1579],
 25: [7],
 36: [290],
 37: [876],
 39: [122],
 41: [817],
 42: [390],
 44: [349, 843],
 45: [1202],
 46: [614, 1409],
 47: [82, 1064],
 48: [687, 715, 1436],
 49: [310, 519, 1219, 1304],
 50: [639],
 51: [863, 917, 1093],
 52: [663, 773],
 53: [969, 1166],
 57: [1270],
 59: [182, 452],
 60: [1367],
 61: [875, 991, 1019, 1321, 1345],
 62: [258, 412],
 67: [41, 102, 364, 571],
 75: [284],
 82: [9],
 83: [617, 1904],
 85: [1827],
 86: [1159, 1493, 1568],
 87: [387],
 88: [229, 435, 1041, 1182, 1591],
 89: [1724, 1779],
 